# Notebook 12: Describing Function
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores ...

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch


# Learning Objectives

After completing this material, you should be able to:

1. **Understand the role of nonlinearities in control systems:**
   - Identify common static (memoryless) nonlinearities such as saturation, dead-zone, quantization, and Schmitt trigger. 
   - Distinguish between static nonlinearities and nonlinearities with memory, such as hysteresis.
   - Explain why linear analysis tools are insufficient for nonlinear systems and how the describing function method addresses this limitation.

2. **Apply the describing function method:**
   - Define the describing function \(N(A)\) for a nonlinear system.
   - Derive the describing function for common nonlinearities, such as saturation and Schmitt triggers.
   - Approximate a nonlinear system using its describing function as an amplitude-dependent gain.

3. **Analyze limit cycles in nonlinear feedback systems:**
   - Recognize how limit cycles can arise in nonlinear feedback systems.
   - Identify limit cycles by locating intersections of the polar plot of \(L(j\omega)\) with \(-1/N(A)\).
   - Predict the amplitude and frequency of limit cycles using the describing function method.

4. **Assess the stability of limit cycles:**
   - Determine the stability of limit cycles using an extension of the Nyquist stability criterion.
   - Identify whether perturbations in amplitude grow or decay, and explain how this determines the stability of a limit cycle.

5. **Apply absolute stability criteria for nonlinear feedback systems:**
   - Understand the concept of absolute stability for a system with one nonlinearity.
   - Use necessary and sufficient conditions for absolute stability, including the circle criterion.
   - Analyze absolute stability using Nyquist plots and encirclements of sectors and circles.


## Required Packages



Run the following cell to import required packages:

In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider, HBox, VBox, Checkbox, interact, Dropdown
from IPython.display import display, clear_output, HTML
from scipy import signal
import control
import warnings

# Motivation

Most real-world systems are **nonlinear**, meaning they do not follow the principle of superposition. Until now, we have handled nonlinearities by **linearizing** around equilibrium points, but this approach fails to capture important global behaviors like **limit cycles**.

To address this, we introduce the concept of **describing functions**, which approximate nonlinear elements as **amplitude-dependent gains**. This method allows us to predict the existence of **limit cycles**, analyze their stability, and check for **absolute stability** in feedback systems. By the end of this lecture, you will be able to apply describing functions to analyze and understand nonlinear control systems.

## Common Nonlinearities

Nonlinear elements are present in many real-world control systems. Below are some of the most common types of nonlinearities, along with real-life examples for each:

1. **Saturation**: When a system's output is limited to a maximum or minimum value.
   - **Example**: The throttle of a car can only open between 0% and 100%, regardless of the driver's input.

2. **Dead Zone**: A region where small inputs produce no output change.
   - **Example**: A joystick in a game controller may have a small range around the neutral position where movement does not affect the game.

3. **Quantization**: Continuous signals are mapped to discrete levels.
   - **Example**: A digital thermometer rounds continuous temperature measurements to the nearest integer.

4. **Hysteresis**: The output depends on the input's history, not just its current value.
   - **Example**: A thermostat for home heating switches on below one temperature but only switches off after exceeding a higher temperature.

5. **Relay (Switching Nonlinearity)**: The output switches between two distinct levels.
   - **Example**: An on-off controller for a streetlight that turns on at dusk and off at dawn.

These nonlinearities often arise in practical systems and can significantly affect system behavior, making them essential to understand for control design.


In the following plot, you can see how different nonlinearities shape the response to a sinusoidal input. Adjust the amplitude \(A\) and select a nonlinearity to observe its effect on the system's output.

In [ ]:
def plot_nonlinearity(nonlinearity_type='saturation', A=1.5):
    """
    Plots the response of common nonlinear elements to a sinusoidal input.
    
    Parameters:
    - nonlinearity_type: The type of nonlinearity ('saturation', 'dead_zone', 'quantizer', 'hysteresis', 'relay').
    - A: Amplitude of the sinusoidal input.
    """
    t = np.linspace(0, 2 * np.pi, 1000)  # Time vector
    u = A * np.sin(t)  # Sinusoidal input
    
    if nonlinearity_type == 'saturation':
        y = np.clip(u, -1, 1)  # Saturation limits the output to [-1, 1]
    elif nonlinearity_type == 'dead_zone':
        dead_zone_width = 0.2  # Width of the dead zone
        y = np.where(np.abs(u) < dead_zone_width, 0, u)  # Zero output in the dead zone
    elif nonlinearity_type == 'quantizer':
        step_size = 0.5  # Size of quantization step
        y = np.round(u / step_size) * step_size  # Round input to the nearest multiple of step_size
    elif nonlinearity_type == 'hysteresis':
        y = np.zeros_like(u)
        state = 0  # Start at zero state
        for i in range(1, len(u)):
            if u[i] > 0.2:
                state = 1
            elif u[i] < -0.2:
                state = -1
            y[i] = state  # Hysteresis holds its state
    elif nonlinearity_type == 'relay':
        relay_threshold = 0.2
        y = np.where(u > relay_threshold, 1, np.where(u < -relay_threshold, -1, 0))  # Relay switch
    else:
        y = u  # Default to no nonlinearity (identity)
    
    plt.figure(figsize=(8, 4))
    plt.plot(t, u, label='Input $u(t)$', linestyle='dashed', color='blue')
    plt.plot(t, y, label='Output $y(t)$', color='red')
    plt.xlabel('Time (t)')
    plt.ylabel('Amplitude')
    plt.title(f'Nonlinearity Type: {nonlinearity_type.capitalize()} (A = {A})')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.legend()
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.show()

# Interactive widget to change nonlinearity type and input amplitude
interact(plot_nonlinearity, 
         nonlinearity_type=Dropdown(options=['saturation', 'dead_zone', 'quantizer', 'hysteresis', 'relay'], 
                                    value='saturation', 
                                    description='Nonlinearity:'),
         A=FloatSlider(min=0.5, max=3, step=0.1, value=1.5, description='Amplitude (A)'))